First updated @ Nov 2, 2021

Last updated @ Nov 9, 2021

In [1]:
import pandas as pd
import ast
from collections import defaultdict
import numpy as np

## Load 3 datasets

- The Bechdel data (only has imdb id)
- The credits data (only has tmdb id)
- The links data, which has both imdb id and tmdb id, so that we can join the three datasets together to get the movies we want.

In [2]:
# load bechdel data
bechdel_df = pd.read_csv('./Data/Bechdel_detailed.csv')
bechdel_df.rename(columns={'rating': 'bt_score'}, inplace=True) # I find the name rating a little confusing
bechdel_df

,title,year,bt_score,dubious,imdbid,id,submitterid,date,visible
0,Passage de Venus,1874.0,0.0,0.0,3155794.0,9602.0,18880.0,2021-04-02 20:58:09,1.0
1,La Rosace Magique,1877.0,0.0,0.0,14495706.0,9804.0,19145.0,2021-05-11 00:11:22,1.0
2,Sallie Gardner at a Gallop,1878.0,0.0,0.0,2221420.0,9603.0,18882.0,2021-04-03 02:25:27,1.0
3,Le singe musicien,1878.0,0.0,0.0,12592084.0,9806.0,19151.0,2021-05-11 23:38:54,1.0
4,Athlete Swinging a Pick,1881.0,0.0,0.0,7816420.0,9816.0,19162.0,2021-05-13 01:32:14,1.0
...,...,...,...,...,...,...,...,...,...
9324,"Guilty, The",2021.0,1.0,0.0,9421570.0,10079.0,19599.0,2021-10-10 20:16:13,1.0
9325,Till Death,2021.0,0.0,0.0,11804152.0,10083.0,19613.0,2021-10-18 19:58:48,1.0
9326,"Last Duel, The",2021.0,3.0,1.0,4244994.0,10085.0,19619.0,2021-10-20 02:30:29,1.0
9327,Venom Let there be Carnage,2021.0,2.0,0.0,7097896.0,10092.0,19627.0,2021-10-21 10:11:03,1.0


In [3]:
# load imdb id and tmdb id data
links_df = pd.read_csv('./TheMoviesData/links.csv', index_col=0)
links_df

,imdbId,tmdbId
movieId,,
1,114709,862.0
2,113497,8844.0
3,113228,15602.0
4,114885,31357.0
5,113041,11862.0
...,...,...
176269,6209470,439050.0
176271,2028550,111109.0
176273,303758,67758.0


In [4]:
# Load movie credits data
credits_df = pd.read_csv('./TheMoviesData/credits.csv')
credits_df.head()

,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602
3,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862


## Inner join 3 datasets and clean up empty values

In [5]:
bechdel_df = pd.merge(bechdel_df, links_df, left_on='imdbid', right_on='imdbId', how='inner').drop(columns=['imdbId'])
bechdel_df = pd.merge(bechdel_df, credits_df, left_on='tmdbId', right_on='id', how='inner').drop(columns=['imdbid', 'submitterid', 'date', 'visible', 'tmdbId', 'id_x', 'id_y'])
bechdel_df

,title,year,bt_score,dubious,cast,crew
0,Passage de Venus,1874.0,0.0,0.0,[],"[{'credit_id': '55325dc09251417ae30009c3', 'de..."
1,Sallie Gardner at a Gallop,1878.0,0.0,0.0,"[{'cast_id': 5, 'character': 'Herself - Sallie...","[{'credit_id': '52fe4ce29251416c91103327', 'de..."
2,Buffalo Running,1883.0,0.0,0.0,[],"[{'credit_id': '5831b973c3a3685b960216b5', 'de..."
3,Man Walking Around the Corner,1887.0,0.0,0.0,[],"[{'credit_id': '52fe4c239251416c910f132b', 'de..."
4,Accordion Player,1888.0,0.0,0.0,"[{'cast_id': 1, 'character': 'Himself', 'credi...","[{'credit_id': '52fe49c39251416c750d2a8b', 'de..."
...,...,...,...,...,...,...
7610,Landline,2017.0,3.0,0.0,"[{'cast_id': 0, 'character': 'Dana Jacobs', 'c...","[{'credit_id': '5863a7fcc3a3680ab600716c', 'de..."
7611,Patient Zero,2018.0,1.0,0.0,"[{'cast_id': 1, 'character': 'Dr. Gina Rose', ...","[{'credit_id': '55c8fd47c3a36849b40002ec', 'de..."
7612,Iron Sky: The Coming Race,2019.0,3.0,0.0,"[{'cast_id': 1, 'character': 'Wolfgang Kortzfl...","[{'credit_id': '5461f666c3a3686f4c001f27', 'de..."
7613,Ip Man 4: The Finale,2019.0,3.0,0.0,"[{'cast_id': 0, 'character': '', 'credit_id': ...",[]


After merging the three dataframes, we can already see a lot of movies being dropped because of missing information.

To make things worse, we can see plenty of empty values in cast and crew... not good. Let's calculate the percentage of empty values and see what we should do.

In [6]:
count1, count2 = 0, 0
for index, row in bechdel_df.iterrows():
    cast_list = ast.literal_eval(row['cast'])
    crew_list = ast.literal_eval(row['crew'])
    if len(cast_list) == 0:
        count1 += 1
    if len(crew_list) == 0:
        count2 += 1
print('Empty cast percentage:', str('{:.2f}'.format(count1/len(bechdel_df) * 100))+'%')
print('Empty crew percentage:', str('{:.2f}'.format(count2/len(bechdel_df) * 100))+'%')

Empty cast percentage: 0.39%
Empty crew percentage: 0.11%


This is not bad at all! Let's drop them then.

In [7]:
bechdel_df = bechdel_df[(bechdel_df['cast'] != '[]') & (bechdel_df['crew'] != '[]')].reset_index()
bechdel_df

,index,title,year,bt_score,dubious,cast,crew
0,1,Sallie Gardner at a Gallop,1878.0,0.0,0.0,"[{'cast_id': 5, 'character': 'Herself - Sallie...","[{'credit_id': '52fe4ce29251416c91103327', 'de..."
1,4,Accordion Player,1888.0,0.0,0.0,"[{'cast_id': 1, 'character': 'Himself', 'credi...","[{'credit_id': '52fe49c39251416c750d2a8b', 'de..."
2,6,"Monkeyshines, No. 1",1890.0,0.0,0.0,"[{'cast_id': 10, 'character': 'Himself', 'cred...","[{'credit_id': '52fe44d99251416c9101ef9f', 'de..."
3,7,"Monkeyshines, No. 2",1890.0,0.0,0.0,"[{'cast_id': 1, 'character': 'Himself', 'credi...","[{'credit_id': '52fe450c9251416c91025b0b', 'de..."
4,9,"Monkeyshines, No. 3",1890.0,0.0,0.0,"[{'cast_id': 1, 'character': 'Himself', 'credi...","[{'credit_id': '52fe450c9251416c91025b2f', 'de..."
...,...,...,...,...,...,...,...
7574,7609,12 feet deep,2017.0,3.0,0.0,"[{'cast_id': 0, 'character': 'McGradey', 'cred...","[{'credit_id': '592ec68d92514130da00d716', 'de..."
7575,7610,Landline,2017.0,3.0,0.0,"[{'cast_id': 0, 'character': 'Dana Jacobs', 'c...","[{'credit_id': '5863a7fcc3a3680ab600716c', 'de..."
7576,7611,Patient Zero,2018.0,1.0,0.0,"[{'cast_id': 1, 'character': 'Dr. Gina Rose', ...","[{'credit_id': '55c8fd47c3a36849b40002ec', 'de..."
7577,7612,Iron Sky: The Coming Race,2019.0,3.0,0.0,"[{'cast_id': 1, 'character': 'Wolfgang Kortzfl...","[{'credit_id': '5461f666c3a3686f4c001f27', 'de..."


In [8]:
bechdel_df.groupby('year')['bt_score'].count().to_frame().tail(20)
# strange that we don't have recent movies anymore...
# Having more recent movies might have been more informative
# but let's analyze what we have

,bt_score
year,
2001.0,164
2002.0,175
2003.0,158
2004.0,186
2005.0,195
2006.0,223
2007.0,238
2008.0,246
2009.0,309


## Extract cast & crew Gender Info

Since there are too many unknowns, I am going to predict the gender from first names. 

Package: https://pypi.org/project/gender-guesser/

In [9]:
# uncomment the line below and run if you haven't installed the gender prediction package
# %pip install gender-guesser

In [10]:
import gender_guesser.detector as gender
d = gender.Detector()

In [11]:
gender_df = pd.DataFrame(columns=[
    'cast_total_num', 
    'cast_female_num', 
    'cast_male_num', 
    'cast_unknown_num',
    'crew_total_num', 
    'crew_female_num', 
    'crew_male_num',
    'crew_unknown_num',
    'Directing_total', 
    'Directing_female',
    'Directing_male',
    'Directing_unknown',
    'Writing_total',
    'Writing_female',
    'Writing_male',
    'Writing_unknown',
    'Production_total',
    'Production_female',
    'Production_male',
    'Production_unknown',
    ])
gender_df.head()

,cast_total_num,cast_female_num,cast_male_num,cast_unknown_num,crew_total_num,crew_female_num,crew_male_num,crew_unknown_num,Directing_total,Directing_female,Directing_male,Directing_unknown,Writing_total,Writing_female,Writing_male,Writing_unknown,Production_total,Production_female,Production_male,Production_unknown


In [12]:
overview_info = defaultdict(int)

for index, row in bechdel_df.iterrows():
    cast_info, crew_info, crew_department = defaultdict(int), defaultdict(int), defaultdict(int)
    cast_list = ast.literal_eval(row['cast'])
    crew_list = ast.literal_eval(row['crew'])

    # count everything in cast
    for cast in cast_list:
        if cast['gender'] == 2: # male
            cast_info['male'] += 1
        elif cast['gender'] == 1: # female
            cast_info['female'] += 1
        elif cast['gender'] == 0: # unknown gender so we predict from first name
            first_name = cast['name'].split(' ')[0]
            predicted_gender = d.get_gender(first_name)
            if predicted_gender == 'male' or predicted_gender == 'mostly_male':
                cast_info['male'] += 1
            elif predicted_gender == 'female' or predicted_gender == 'mostly_female':
                cast_info['female'] += 1
            elif predicted_gender == 'unknown' or predicted_gender == 'andy':
                cast_info['unknown'] += 1
        else:
            print(f'Oops1 index{index} is unusal: {cast}')
    gender_df.at[index, 'cast_total_num'] = len(cast_list)
    gender_df.at[index, 'cast_female_num'] = cast_info['female']
    gender_df.at[index, 'cast_male_num'] = cast_info['male']
    gender_df.at[index, 'cast_unknown_num'] = cast_info['unknown']

    # count evetything in crew
    for crew in crew_list:
        if crew['gender'] == 2:
            crew_info['male'] += 1
            crew_department[str(crew['department'])+'_male'] += 1
            crew_department[str(crew['department'])+'_total'] += 1
        elif crew['gender'] == 1:
            crew_info['female'] += 1
            crew_department[str(crew['department'])+'_female'] += 1
            crew_department[str(crew['department'])+'_total'] += 1
        elif crew['gender'] == 0:
            first_name = crew['name'].split(' ')[0]
            predicted_gender = d.get_gender(first_name)
            if predicted_gender == 'male' or predicted_gender == 'mostly_male':
                crew_info['male'] += 1
                crew_department[str(crew['department'])+'_male'] += 1
            elif predicted_gender == 'female' or predicted_gender == 'mostly_female':
                crew_info['female'] += 1
                crew_department[str(crew['department'])+'_female'] += 1
            elif predicted_gender == 'unknown' or predicted_gender == 'andy':
                crew_info['unknown'] += 1
                crew_department[str(crew['department'])+'_unknown'] += 1
            crew_department[str(crew['department'])+'_total'] += 1
        else:
            print(f'Oops2 index{index} is unusal: {crew}')
    gender_df.at[index, 'crew_total_num'] = len(crew_list)
    gender_df.at[index, 'crew_female_num'] = crew_info['female']
    gender_df.at[index, 'crew_male_num'] = crew_info['male']
    gender_df.at[index, 'crew_unknown_num'] = crew_info['unknown']

    # aggregate all counts to overview data
    overview_info['cast_total'] += len(cast_list)
    for cast_gender, count in cast_info.items():
        overview_info['cast_'+str(cast_gender)] += count
    overview_info['crew_total'] += len(crew_list)
    for crew_gender, count in crew_info.items():
        overview_info['crew_'+str(crew_gender)] += count
    for department, count in crew_department.items():
        overview_info[department] += count
        gender_df.at[index, department] = count
gender_df = gender_df.fillna(0)

In [13]:
gender_df.tail(10)

,cast_total_num,cast_female_num,cast_male_num,cast_unknown_num,crew_total_num,crew_female_num,crew_male_num,crew_unknown_num,Directing_total,Directing_female,...,Lighting_total,Visual Effects_unknown,Visual Effects_female,Lighting_unknown,Camera_female,Actors_female,Actors_total,Lighting_female,Actors_male,Actors_unknown
7569,37,10,26,1,33,11,21,1,2,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7570,54,9,41,4,72,20,48,4,2,1,...,3.0,0.0,3.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0
7571,11,5,6,0,7,1,6,0,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7572,7,2,5,0,5,0,5,0,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7573,7,4,2,1,6,1,4,1,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7574,8,3,4,1,71,19,50,2,5,1,...,5.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7575,18,7,10,1,26,19,6,1,2,2,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7576,16,8,8,0,6,0,6,0,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7577,30,7,21,2,5,1,3,1,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7578,3,0,3,0,7,0,7,0,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [14]:
print('Percentage of unknowns in Cast:', str('{:.2f}'.format(overview_info['cast_unknown']/overview_info['cast_total'] * 100))+'%')
print('Percentage of unknowns in Crew:', '{:.2f}'.format(overview_info['crew_unknown']/overview_info['crew_total'] * 100)+'%')
print('Percentage of unknowns in Directing:', str('{:.2f}'.format(overview_info['Directing_unknown']/overview_info['Directing_total'] * 100))+'%')
print('Percentage of unknowns in Writing:', str('{:.2f}'.format(overview_info['Writing_unknown']/overview_info['Writing_total'] * 100))+'%')
print('Percentage of unknowns in Production:', str('{:.2f}'.format(overview_info['Production_unknown']/overview_info['Production_total'] * 100))+'%')

Percentage of unknowns in Cast: 4.71%
Percentage of unknowns in Crew: 4.93%
Percentage of unknowns in Directing: 4.19%
Percentage of unknowns in Writing: 4.86%
Percentage of unknowns in Production: 4.18%


I'd say this amount of unknown genders is perfectly okay.

Next, we need to think about what gender information we really need.

What related closely to female representation can be:
- Cast female ratio
- Crew female ratio
- Director female ratio
- Writer female ratio
- Producer female ratio

In [17]:
gender_df.columns

Index(['cast_total_num', 'cast_female_num', 'cast_male_num',
       'cast_unknown_num', 'crew_total_num', 'crew_female_num',
       'crew_male_num', 'crew_unknown_num', 'Directing_total',
       'Directing_female', 'Directing_male', 'Directing_unknown',
       'Writing_total', 'Writing_female', 'Writing_male', 'Writing_unknown',
       'Production_total', 'Production_female', 'Production_male',
       'Production_unknown', 'Crew_unknown', 'Crew_total', 'Camera_male',
       'Camera_total', 'Crew_male', 'Art_male', 'Art_total', 'Sound_male',
       'Sound_total', 'Editing_male', 'Editing_total', 'Art_unknown',
       'Costume & Make-Up_female', 'Costume & Make-Up_total', 'Camera_unknown',
       'Editing_female', 'Costume & Make-Up_unknown', 'Costume & Make-Up_male',
       'Visual Effects_male', 'Visual Effects_total', 'Sound_female',
       'Crew_female', 'Editing_unknown', 'Art_female', 'Sound_unknown',
       'Lighting_male', 'Lighting_total', 'Visual Effects_unknown',
       'Visua

In [20]:
bechdel_df['cast_female_ratio'] = gender_df['cast_female_num'] / gender_df['cast_total_num']
bechdel_df['crew_female_ratio'] = gender_df['crew_female_num'] / gender_df['crew_total_num']
bechdel_df['directing_female_ratio'] = gender_df['Directing_female'] / gender_df['Directing_total']
bechdel_df['writing_female_ratio'] = gender_df['Writing_female'] / gender_df['Writing_total']
bechdel_df['production_female_ratio'] = gender_df['Production_female'] / gender_df['Production_total']
bechdel_df['actors_female_ratio'] = gender_df['Actors_female'] / gender_df['Actors_total']
bechdel_df

,index,title,year,bt_score,dubious,cast,crew,cast_female_ratio,crew_female_ratio,directing_female_ratio,writing_female_ratio,production_female_ratio,actors_female_ratio
0,1,Sallie Gardner at a Gallop,1878.0,0.0,0.0,"[{'cast_id': 5, 'character': 'Herself - Sallie...","[{'credit_id': '52fe4ce29251416c91103327', 'de...",0.500000,0.000000,0.0,NaN,0.000000,NaN
1,4,Accordion Player,1888.0,0.0,0.0,"[{'cast_id': 1, 'character': 'Himself', 'credi...","[{'credit_id': '52fe49c39251416c750d2a8b', 'de...",0.000000,0.000000,0.0,NaN,NaN,NaN
2,6,"Monkeyshines, No. 1",1890.0,0.0,0.0,"[{'cast_id': 10, 'character': 'Himself', 'cred...","[{'credit_id': '52fe44d99251416c9101ef9f', 'de...",0.000000,0.000000,0.0,NaN,0.000000,NaN
3,7,"Monkeyshines, No. 2",1890.0,0.0,0.0,"[{'cast_id': 1, 'character': 'Himself', 'credi...","[{'credit_id': '52fe450c9251416c91025b0b', 'de...",0.000000,0.000000,0.0,NaN,NaN,NaN
4,9,"Monkeyshines, No. 3",1890.0,0.0,0.0,"[{'cast_id': 1, 'character': 'Himself', 'credi...","[{'credit_id': '52fe450c9251416c91025b2f', 'de...",0.000000,0.000000,0.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7574,7609,12 feet deep,2017.0,3.0,0.0,"[{'cast_id': 0, 'character': 'McGradey', 'cred...","[{'credit_id': '592ec68d92514130da00d716', 'de...",0.375000,0.267606,0.2,0.00,0.473684,NaN
7575,7610,Landline,2017.0,3.0,0.0,"[{'cast_id': 0, 'character': 'Dana Jacobs', 'c...","[{'credit_id': '5863a7fcc3a3680ab600716c', 'de...",0.388889,0.730769,1.0,1.00,0.733333,NaN
7576,7611,Patient Zero,2018.0,1.0,0.0,"[{'cast_id': 1, 'character': 'Dr. Gina Rose', ...","[{'credit_id': '55c8fd47c3a36849b40002ec', 'de...",0.500000,0.000000,0.0,0.00,0.000000,NaN
7577,7612,Iron Sky: The Coming Race,2019.0,3.0,0.0,"[{'cast_id': 1, 'character': 'Wolfgang Kortzfl...","[{'credit_id': '5461f666c3a3686f4c001f27', 'de...",0.233333,0.200000,0.0,0.25,NaN,NaN


In [24]:
bechdel_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7579 entries, 0 to 7578
Data columns (total 13 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   index                    7579 non-null   int64  
 1   title                    7579 non-null   object 
 2   year                     7579 non-null   float64
 3   bt_score                 7579 non-null   float64
 4   dubious                  7281 non-null   float64
 5   cast                     7579 non-null   object 
 6   crew                     7579 non-null   object 
 7   cast_female_ratio        7579 non-null   float64
 8   crew_female_ratio        7579 non-null   float64
 9   directing_female_ratio   7575 non-null   float64
 10  writing_female_ratio     7318 non-null   float64
 11  production_female_ratio  6135 non-null   float64
 12  actors_female_ratio      5 non-null      float64
dtypes: float64(9), int64(1), object(3)
memory usage: 769.9+ KB


In [22]:
bechdel_df.isna().sum() # why are there so many nan's in dubious????????? There should not be. 

index                         0
title                         0
year                          0
bt_score                      0
dubious                     298
cast                          0
crew                          0
cast_female_ratio             0
crew_female_ratio             0
directing_female_ratio        4
writing_female_ratio        261
production_female_ratio    1444
actors_female_ratio        7574
dtype: int64